In [1]:
CapacityMetricsWorkspace = 'Microsoft Fabric Capacity Metrics'
CapacityMetricsDataset = 'Fabric Capacity Metrics'

StatementMeta(, 1ed15bb0-9999-46e0-bef6-023897e4f339, 3, Finished, Available, Finished)

### Get the workspace id

In [2]:
workspaceId = spark.conf.get('trident.workspace.id')

StatementMeta(, 1ed15bb0-9999-46e0-bef6-023897e4f339, 4, Finished, Available, Finished)

In [3]:
import requests, json

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
          }

response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}', headers=header)
workspaceName = response.json().get('displayName')
print(workspaceName)

StatementMeta(, 1ed15bb0-9999-46e0-bef6-023897e4f339, 5, Finished, Available, Finished)

WS_DW_Query_Cost_Test_Take_Four


### Create lakehouses

In [4]:
from notebookutils import mssparkutils

for lakehouseName in ['LH_SampleData', 'LH_QueryResults']:
    try:
        lakehouseItem = mssparkutils.lakehouse.create(lakehouseName, "", workspaceId)
    except Exception as e:
        # print(e)
        pass

LH_SampleData = mssparkutils.lakehouse.get("LH_SampleData", workspaceId)
LH_QueryResults = mssparkutils.lakehouse.get("LH_QueryResults", workspaceId)
print(LH_SampleData, '\n', LH_QueryResults)

StatementMeta(, 1ed15bb0-9999-46e0-bef6-023897e4f339, 6, Finished, Available, Finished)

{'id': 'f84b1c16-4313-4b1a-a61f-5fb52c3196bd', 'type': 'Lakehouse', 'displayName': 'LH_SampleData', 'description': '', 'workspaceId': '0aac7c1d-607f-4f30-aa84-bfc4e545a8b2', 'properties': {'abfsPath': 'abfss://0aac7c1d-607f-4f30-aa84-bfc4e545a8b2@onelake.dfs.fabric.microsoft.com/f84b1c16-4313-4b1a-a61f-5fb52c3196bd'}} 
 {'id': '12e704fb-7b0e-4fba-ad59-4f2e072501b4', 'type': 'Lakehouse', 'displayName': 'LH_QueryResults', 'description': '', 'workspaceId': '0aac7c1d-607f-4f30-aa84-bfc4e545a8b2', 'properties': {'abfsPath': 'abfss://0aac7c1d-607f-4f30-aa84-bfc4e545a8b2@onelake.dfs.fabric.microsoft.com/12e704fb-7b0e-4fba-ad59-4f2e072501b4'}}


### Get notebook definition from github

In [5]:
import requests, json, base64
url = "https://raw.githubusercontent.com/bretamyers/FabricTools/dev/FabricDWQueryCostAnalyzer/NB_DW_Load_Cost_Analyzer.ipynb"
response = requests.get(url)

b = base64.b64encode(bytes(json.dumps(response.json()), 'utf-8')) # bytes
base64_str = b.decode('utf-8') # convert bytes to string

StatementMeta(, 1ed15bb0-9999-46e0-bef6-023897e4f339, 7, Finished, Available, Finished)

### Create notebook from the definition

In [6]:
import requests, json, time, base64

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
          }

body = {
    "displayName": "NB_DW_Load_Cost_Analyzer",
    "description": "A notebook description",
    "metadata": {
          "dependencies": {
              "lakehouse": {
                  "default_lakehouse": LH_QueryResults.get('id'),
                  "default_lakehouse_workspace_id": workspaceId
              }
          }
      },
    "definition": {
        "format": "ipynb",
        "parts": [
            {
                "path": "notebook-content.py"
                ,"payload": base64_str
                ,"payloadType": "InlineBase64"
            }
        ]
    }
}
# print(json.dumps(body))
response = requests.request(method='post', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/notebooks', headers=header, data=json.dumps(body))
# print(response.text)
# print(response.json())
# print(response.headers)

if response.status_code == 202:
    time.sleep(int(response.headers.get('Retry-After')))
    for retry in range(5):
        responseStatus = requests.request(method='get', url=response.headers.get('Location'), headers=header)
        print(responseStatus.text)
        print(responseStatus.headers)
        if responseStatus.headers.get('Location') is not None:
            print(responseStatus.text)
            print(responseStatus.json())
            print(responseStatus.headers)
            break
        else:
            time.sleep(int(responseStatus.headers.get('Retry-After')))
else:
    print(f"Error - {response.json()}")

StatementMeta(, 1ed15bb0-9999-46e0-bef6-023897e4f339, 8, Finished, Available, Finished)

{"status":"Succeeded","createdTimeUtc":"2024-09-23T19:11:29.7641986","lastUpdatedTimeUtc":"2024-09-23T19:11:30.8735946","percentComplete":100,"error":null}
{'Cache-Control': 'no-store, must-revalidate, no-cache', 'Pragma': 'no-cache', 'Content-Length': '132', 'Content-Type': 'application/json; charset=utf-8', 'Content-Encoding': 'gzip', 'Location': 'https://api.fabric.microsoft.com/v1/operations/b3f584e8-247a-4d56-8ad1-d9e01b158f2e/result', 'x-ms-operation-id': 'b3f584e8-247a-4d56-8ad1-d9e01b158f2e', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'X-Frame-Options': 'deny', 'X-Content-Type-Options': 'nosniff', 'RequestId': '8abe5fa6-17a9-41cb-b21c-6ff9b1222212', 'Access-Control-Expose-Headers': 'RequestId,Location,x-ms-operation-id', 'request-redirected': 'true', 'home-cluster-uri': 'https://wabi-west-us3-a-primary-redirect.analysis.windows.net/', 'Date': 'Mon, 23 Sep 2024 19:11:50 GMT'}
{"status":"Succeeded","createdTimeUtc":"2024-09-23T19:11:29.7641986","lastUpdat

### Get the notebook id

In [7]:
import requests, json

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
          }
          
# Need to implement for pagination
response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/notebooks', headers=header)

for notebook in response.json().get('value'):
    if notebook.get('displayName') == 'NB_DW_Load_Cost_Analyzer':
        notebookId = notebook.get('id')
        print(notebookId)


StatementMeta(, 1ed15bb0-9999-46e0-bef6-023897e4f339, 9, Finished, Available, Finished)

2573832d-f426-4975-9acf-3978c3963319


### Create Warehouse

In [8]:
import requests, json, time

body = {"displayName": "WH_SampleData"}

response = requests.request(method='post', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/warehouses', headers=header, data=json.dumps(body))

if response.status_code == 202:
    print(f'Creating Warehouse {body.get("displayName")}')
    time.sleep(int(response.headers.get('Retry-After')))
    for retry in range(5):
        responseStatus = requests.request(method='get', url=response.headers.get('Location'), headers=header)
        print(responseStatus.text)
        print(responseStatus.headers)
        if responseStatus.json().get('status') != 'Succeeded':
            time.sleep(int(responseStatus.headers.get('Retry-After')))
        else:
            if responseStatus.headers.get('Location') is None: # no result was return but operation completed
                break
            else:
                responseResult = requests.request(method='get', url=responseStatus.headers.get('Location'), headers=header)
                print(f"Succeeded {responseResult.json()}")
                break
else:
    print(response.text)

StatementMeta(, 1ed15bb0-9999-46e0-bef6-023897e4f339, 10, Finished, Available, Finished)

Creating Warehouse WH_SampleData
{"status":"Succeeded","createdTimeUtc":"2024-09-23T19:11:54.4052071","lastUpdatedTimeUtc":"2024-09-23T19:12:07.2022855","percentComplete":100,"error":null}
{'Cache-Control': 'no-store, must-revalidate, no-cache', 'Pragma': 'no-cache', 'Content-Length': '131', 'Content-Type': 'application/json; charset=utf-8', 'Content-Encoding': 'gzip', 'Location': 'https://api.fabric.microsoft.com/v1/operations/5f66293c-8ed7-49be-987c-6212dc088df7/result', 'x-ms-operation-id': '5f66293c-8ed7-49be-987c-6212dc088df7', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'X-Frame-Options': 'deny', 'X-Content-Type-Options': 'nosniff', 'RequestId': 'eb14a82b-599f-433c-acb5-c86252cabd63', 'Access-Control-Expose-Headers': 'RequestId,Location,x-ms-operation-id', 'request-redirected': 'true', 'home-cluster-uri': 'https://wabi-west-us3-a-primary-redirect.analysis.windows.net/', 'Date': 'Mon, 23 Sep 2024 19:12:13 GMT'}
Succeeded {'id': '9f35088d-d1c1-46e9-be5d-bec4

In [9]:
import requests, json, time

response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/warehouses', headers=header)

for warehouse in response.json().get('value'):
    if warehouse.get('displayName') == 'WH_SampleData':
        WH_SampleData = warehouse
        # response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/warehouses/{_.get("id")}', headers=header)
print(WH_SampleData)

StatementMeta(, 1ed15bb0-9999-46e0-bef6-023897e4f339, 11, Finished, Available, Finished)

{'id': '9f35088d-d1c1-46e9-be5d-bec4b55aaf17', 'type': 'Warehouse', 'displayName': 'WH_SampleData', 'description': '', 'workspaceId': '0aac7c1d-607f-4f30-aa84-bfc4e545a8b2', 'properties': {'connectionInfo': 'n52dzjnhphme5jslxpytuo62ni-dv6kyct7maye7kuex7cokrniwi.datawarehouse.fabric.microsoft.com', 'connectionString': 'n52dzjnhphme5jslxpytuo62ni-dv6kyct7maye7kuex7cokrniwi.datawarehouse.fabric.microsoft.com', 'createdDate': '2024-09-23T19:11:53.87', 'lastUpdatedTime': '2024-09-23T19:12:07.01'}}


### Load data to Lakehouse

In [10]:
import azure.storage.blob
import pandas as pd
from notebookutils import mssparkutils

spark.conf.set('spark.sql.parquet.int96RebaseModeInWrite', 'LEGACY')

account_url = "https://fabrictutorialdata.blob.core.windows.net"
blob_service_client = azure.storage.blob.BlobServiceClient(account_url)
container_client = blob_service_client.get_container_client('sampledata')

tableList = list()
for blob in container_client.walk_blobs(name_starts_with='WideWorldImportersDW/parquet/tables/', delimiter='/'):
    tableName = blob.name.split('/')[-1].split('.parquet')[0]
    tableList.append(tableName)

for table in tableList:
    print(f'Loading Table: {table}')
    if mssparkutils.fs.exists(f"{LH_SampleData.get('properties').get('abfsPath')}/Tables/{table}"):
        mssparkutils.fs.rm(f"{LH_SampleData.get('properties').get('abfsPath')}/Tables/{table}", recurse=True)

    for blob in container_client.walk_blobs(name_starts_with=f"WideWorldImportersDW/parquet/tables/", delimiter='/'):
        if f'{table}.parquet' == blob.name.split('/')[-1]:
            dfPandas = pd.read_parquet(f'{account_url}/sampledata/{blob.name}', engine='pyarrow', storage_options={'anon': True})

            if not dfPandas.empty:
                dfSpark = spark.createDataFrame(dfPandas)
                dfSpark.write.format('delta').mode('append').option('mergeSchema', "true").save(f"{LH_SampleData.get('properties').get('abfsPath')}/Tables/{table}")
            else:
                print('Bad file', blob.name)

StatementMeta(, 1ed15bb0-9999-46e0-bef6-023897e4f339, 12, Finished, Available, Finished)

Loading Table: DimCity
Loading Table: DimCustomer
Loading Table: DimDate
Loading Table: DimEmployee
Loading Table: DimPaymentMethod
Loading Table: DimStockItem
Loading Table: DimSupplier
Loading Table: DimTransactionType
Loading Table: FactMovement
Loading Table: FactOrder
Loading Table: FactPurchase
Loading Table: FactSale
Loading Table: FactStockHolding
Loading Table: FactTransaction


### Load data to Warehouse

In [16]:
from notebookutils import mssparkutils  
from pyspark.sql import functions as F
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, TimestampType, DoubleType
import pyodbc, struct, itertools, time, datetime, re, uuid, json

connectionString = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={WH_SampleData.get("properties").get("connectionString")};Database={WH_SampleData.get("displayName")}'

# Use the credentials of the user executing the notebook
token = bytes(mssparkutils.credentials.getToken('pbi'), "UTF-8")
encoded_bytes = bytes(itertools.chain.from_iterable(zip(token, itertools.repeat(0))))
tokenstruct = struct.pack("<i", len(encoded_bytes)) + encoded_bytes

def get_result_set(cursor):
    if cursor.description:
        resultList = cursor.fetchall()
        resultColumns = columns = [column[0] for column in cursor.description]
    else:
        resultList = []
        resultColumns = []
    return [dict(zip(resultColumns, [str(col) for col in row])) for row in resultList]

with pyodbc.connect(connectionString, attrs_before = { 1256:tokenstruct }) as conn:
    with conn.cursor() as cursor:
        while retry in range(5):
            cursor.execute(f"""SELECT TABLE_NAME FROM LH_SampleData.INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dbo'""")
            resultSet = get_result_set(cursor)
            if len(resultSet) > 0:
                break
            else:
                time.sleep(10)
        while cursor.nextset():
            queryMessage += ",".join([str(cursor.messages) if cursor.messages else ""])
            resultSet.append(get_result_set(cursor))
        cursor.commit()
        print(json.dumps(resultSet))

    for table in resultSet:
        with conn.cursor() as cursor:
            query = f"""IF OBJECT_ID('dbo.{table.get("TABLE_NAME")}', 'U') IS NOT NULL DROP TABLE dbo.{table.get("TABLE_NAME")}; CREATE TABLE dbo.{table.get("TABLE_NAME")} AS SELECT * FROM LH_SampleData.dbo.{table.get("TABLE_NAME")}"""
            print(query)
            cursor.execute(query)
            cursor.commit()


StatementMeta(, 1ed15bb0-9999-46e0-bef6-023897e4f339, 18, Finished, Available, Finished)

[{"TABLE_NAME": "FactStockHolding"}, {"TABLE_NAME": "FactSale"}, {"TABLE_NAME": "DimCustomer"}, {"TABLE_NAME": "DimSupplier"}, {"TABLE_NAME": "FactMovement"}, {"TABLE_NAME": "DimStockItem"}, {"TABLE_NAME": "FactTransaction"}, {"TABLE_NAME": "DimPaymentMethod"}, {"TABLE_NAME": "FactPurchase"}, {"TABLE_NAME": "DimTransactionType"}, {"TABLE_NAME": "DimEmployee"}, {"TABLE_NAME": "DimCity"}, {"TABLE_NAME": "FactOrder"}, {"TABLE_NAME": "DimDate"}]
IF OBJECT_ID('dbo.FactStockHolding', 'U') IS NOT NULL DROP TABLE dbo.FactStockHolding; CREATE TABLE dbo.FactStockHolding AS SELECT * FROM LH_SampleData.dbo.FactStockHolding
IF OBJECT_ID('dbo.FactSale', 'U') IS NOT NULL DROP TABLE dbo.FactSale; CREATE TABLE dbo.FactSale AS SELECT * FROM LH_SampleData.dbo.FactSale
IF OBJECT_ID('dbo.DimCustomer', 'U') IS NOT NULL DROP TABLE dbo.DimCustomer; CREATE TABLE dbo.DimCustomer AS SELECT * FROM LH_SampleData.dbo.DimCustomer
IF OBJECT_ID('dbo.DimSupplier', 'U') IS NOT NULL DROP TABLE dbo.DimSupplier; CREATE TAB

### Refresh Fabric Capacity Metrics App so that the latest workspace artifacts are in the model

In [12]:
import requests, json, time

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
          }

response = requests.get('https://api.fabric.microsoft.com/v1/workspaces', headers=header)

capacityWorkspaceId = [workspace.get('id') for workspace in response.json().get('value') if workspace.get('displayName') == CapacityMetricsWorkspace][0]
print(f'{capacityWorkspaceId = }')

response = requests.get(f"https://api.powerbi.com/v1.0/myorg/groups/{capacityWorkspaceId}/datasets", headers=header)

datasetId = [dataset.get('id') for dataset in response.json().get('value') if dataset.get('name') == CapacityMetricsDataset][0]
print(f'{datasetId = }')

# tableList = [{"table": "Capacities"}
#             ,{"table": "Workspaces"}
#             ,{"table": "TimePoints"}
#             ,{"table": "TimePointBackgroundDetail"}
#             ,{"table": "Items"}
#             ]
body = {"objects": tableList} # Need to ask Pat what tables are import and what are direct query
# response = requests.post(f"https://api.powerbi.com/v1.0/myorg/groups/{capacityWorkspaceId}/datasets/{datasetId}/refreshes", headers=header, data=json.dumps(body))
response = requests.post(f"https://api.powerbi.com/v1.0/myorg/groups/{capacityWorkspaceId}/datasets/{datasetId}/refreshes", headers=header)

refreshId = response.headers.get('RequestId')
print(f'{refreshId = }')

for attempt in range(12): 
    # https://learn.microsoft.com/en-us/power-bi/connect-data/asynchronous-refresh#get-refreshes
    response = requests.get(f"https://api.powerbi.com/v1.0/myorg/groups/{capacityWorkspaceId}/datasets/{datasetId}/refreshes?$top=1", headers=header)
    if response.json().get('value')[0].get('status') != 'Unknown':
        print(f'Refresh Complete')
        break
    else:
        print(f'Refreshing tables ...')
        time.sleep(10)

StatementMeta(, 1ed15bb0-9999-46e0-bef6-023897e4f339, 14, Finished, Available, Finished)

capacityWorkspaceId = '0315286f-829f-410f-a0aa-1e45af27297c'
datasetId = 'a3443502-d96b-4b3f-99e5-76920052edad'
refreshId = 'e90bb1dc-3e45-4542-accb-34a1f67fc397'
Refreshing tables ...
Refreshing tables ...
Refreshing tables ...
Refreshing tables ...
Refreshing tables ...
Refreshing tables ...
Refresh Complete


### Run the notebook

In [17]:
import requests, json, time

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
          }

body = {
    "executionData": {
        "parameters": {
            "FabricDWWorkspaceName": {"value": workspaceName, "type": "string"}
            ,"FabricDWName": {"value": "WH_SampleData", "type": "string"}
            ,"ConcurrencyNum": {"value": "1", "type": "int"}
            ,"CapacityMetricsWorkspace": {"value": CapacityMetricsWorkspace, "type": "string"}
            ,"CapacityMetricsDataset": {"value": CapacityMetricsDataset, "type": "string"}
        }
    }
}

print(json.dumps(body, indent=4))

response = requests.request(method='post', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/items/{notebookId}/jobs/instances?jobType=RunNotebook', headers=header, data=json.dumps(body))

if response.status_code == 202:
    print('Started Execution of Notebook')
    time.sleep(10) # Sleep for 10 seconds to wait for job to get started. TODO Change in the future to a retry
    print(f"{response.headers=}")
    # jobId = response.headers.get('Location')
    while True:
        print(f"{response.headers.get('Location')=}")
        responseStatus = requests.request(method="get", url=response.headers.get('Location'), headers=header)
        print(f"{responseStatus.json().get('status')=} - {responseStatus.json().get('failureReason') if responseStatus.json().get('failureReason') != None else ''}")
        if responseStatus.json().get('status') in ['Succeeded', 'Failed', 'Completed']:
            print(responseStatus.json().get('status'))
            break
        else:
            time.sleep(120) # Wait 2 minutes to check the status of the run
else:
    print('Error trying to execute')

StatementMeta(, 1ed15bb0-9999-46e0-bef6-023897e4f339, 19, Finished, Available, Finished)

{
    "executionData": {
        "parameters": {
            "FabricDWWorkspaceName": {
                "value": "WS_DW_Query_Cost_Test_Take_Four",
                "type": "string"
            },
            "FabricDWName": {
                "value": "WH_SampleData",
                "type": "string"
            },
            "ConcurrencyNum": {
                "value": "1",
                "type": "int"
            },
            "CapacityMetricsWorkspace": {
                "value": "Microsoft Fabric Capacity Metrics",
                "type": "string"
            },
            "CapacityMetricsDataset": {
                "value": "Fabric Capacity Metrics",
                "type": "string"
            }
        }
    }
}
Started Execution of Notebook
response.headers={'Cache-Control': 'no-store, must-revalidate, no-cache', 'Pragma': 'no-cache', 'Content-Length': '0', 'Content-Type': 'application/octet-stream', 'Content-Encoding': 'gzip', 'Location': 'https://api.fabric.microsoft.com/v

In [14]:
# import requests

# response = requests.request(method="get", url=f'https://api.fabric.microsoft.com/v1/workspaces/a3b40eca-afde-448e-bc54-af3d318657b0/items/d20ea153-dbf1-4821-8243-e823bf95bdbb/jobs/instances/237f073e-f403-4d12-a2ae-63377fbe0335', headers=header)
# print(response.status_code)
# print(response.json())


StatementMeta(, 1ed15bb0-9999-46e0-bef6-023897e4f339, 16, Finished, Available, Finished)